# Create the Excel file for analysis with every table joined

In [1]:
import pandas as pd
import sqlalchemy as sa

from analysis.database import get_engine

## Conditions


In [3]:
with get_engine() as engine:
    conditions_df = pd.read_sql(
        """
    SELECT i.individual_id, c.condition, ic.has_condition
    FROM individual i
    JOIN individual_condition ic
    ON i.individual_id = ic.individual_id
    JOIN condition c
    ON ic.condition_id = c.condition_id
    """,
        engine,
    )

    conditions_df = conditions_df.pivot(index="individual_id",
                                        columns="condition",
                                        values="has_condition")

    # remove the condition column name
    conditions_df.columns.name = None

conditions_df.head()

,Adult-onset primary generalised epilepsy,Arrhythmia at rest,Arrhythmogenic right ventricular cardiomyopathy,Arteriovenous malformation,Ascending aortic aneurysm,Atrial fibrillation,Atrial flutter,Atrial standstill,Atrial tachycardia,Attention deficit hyperactivity disorder,...,Sudden cardiac death,Sudden infant death syndrome,Supraventricular tachycardia,Syncope,Syncope exercise/stress induced,Third-degree atrioventricular block,Unspecified premature ventricular contractions,Ventricular fibrillation,Ventricular tachycardia (unspecified),Weight loss
individual_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False,NaN,...,NaN,NaN,NaN,True,True,NaN,False,NaN,NaN,NaN
4,NaN,False,NaN,NaN,NaN,False,NaN,NaN,True,NaN,...,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
with get_engine() as engine:
    other_condition_info = pd.read_sql(
        """
    SELECT i.individual_id, c.condition, ic.age_of_onset, ic.description, ic.onset_symptoms, ic.age_of_presentation, ic.primary_diagnosis
    FROM individual i
    LEFT JOIN individual_condition ic
    ON i.individual_id = ic.individual_id
    LEFT JOIN condition c
    ON ic.condition_id = c.condition_id
    """,
        engine,
    )

other_condition_info.head()

,individual_id,condition,age_of_onset,description,onset_symptoms,age_of_presentation,primary_diagnosis
0,1,Heart Structure Abnormality,NaN,None,None,NaN,None
1,1,Baseline/resting electrocardiogram abnormality,NaN,None,None,NaN,None
2,1,Polymorphic ventricular tachycardia,NaN,None,None,NaN,None
3,1,Exercise/stress induced polymorphic ventricula...,NaN,None,None,NaN,None
4,2,Heart Structure Abnormality,NaN,None,None,NaN,None


In [23]:
description_and_onset_symptoms = other_condition_info.groupby(
    "individual_id").agg(
    {
        "description": lambda x: "; ".join(x.dropna()),
        "onset_symptoms": lambda x: "; ".join(x.dropna())
    }
)
description_and_onset_symptoms.rename(
    columns={
        "description": "condition_descriptions",
        "onset_symptoms": "condition_onset_symptoms"
    },
    inplace=True
)
description_and_onset_symptoms.head()

,condition_descriptions,condition_onset_symptoms
individual_id,,
1,,
2,,none
3,,
4,,
5,,


In [24]:
# primary diagnosis (whatever is true for the individual)
primary_diagnosis = other_condition_info.groupby("individual_id").apply(
    lambda x: ", ".join(x[x["primary_diagnosis"] == 1]["condition"].values) if
    x["primary_diagnosis"].sum() > 0 else None,
    include_groups=False
)
primary_diagnosis = primary_diagnosis.rename("primary_diagnosis")
primary_diagnosis.head()

individual_id
1    Catecholaminergic polymorphic ventricular tach...
2    Catecholaminergic polymorphic ventricular tach...
3    Catecholaminergic polymorphic ventricular tach...
4    Catecholaminergic polymorphic ventricular tach...
5    Catecholaminergic polymorphic ventricular tach...
Name: primary_diagnosis, dtype: object

In [25]:
age_of_onsets = other_condition_info.pivot(
    index="individual_id",
    columns="condition",
    values=["age_of_onset", "onset_symptoms", "age_of_presentation"],
)
age_of_onsets.columns.name = None
# drop any columns that are all NaN
age_of_onsets = age_of_onsets.dropna(axis=1, how="all")
age_of_onsets.columns = [
    "::".join(col).strip().strip(":") for col in age_of_onsets.columns.values
]

age_of_onsets.head()

,age_of_onset::Catecholaminergic polymorphic ventricular tachycardia 1,onset_symptoms::Catecholaminergic polymorphic ventricular tachycardia 1,age_of_presentation::Catecholaminergic polymorphic ventricular tachycardia 1
individual_id,,,
1,NaN,None,36.0
2,NaN,none,41.0
3,NaN,None,16.0
4,2.0,None,NaN
5,10.0,None,28.0


In [26]:
# join all the tables together
condition_info_df = conditions_df.join(primary_diagnosis).join(
    age_of_onsets).join(description_and_onset_symptoms)

condition_info_df.head()

,Adult-onset primary generalised epilepsy,Arrhythmia at rest,Arrhythmogenic right ventricular cardiomyopathy,Arteriovenous malformation,Ascending aortic aneurysm,Atrial fibrillation,Atrial flutter,Atrial standstill,Atrial tachycardia,Attention deficit hyperactivity disorder,...,Unspecified premature ventricular contractions,Ventricular fibrillation,Ventricular tachycardia (unspecified),Weight loss,primary_diagnosis,age_of_onset::Catecholaminergic polymorphic ventricular tachycardia 1,onset_symptoms::Catecholaminergic polymorphic ventricular tachycardia 1,age_of_presentation::Catecholaminergic polymorphic ventricular tachycardia 1,condition_descriptions,condition_onset_symptoms
individual_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Catecholaminergic polymorphic ventricular tach...,NaN,None,36.0,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,Catecholaminergic polymorphic ventricular tach...,NaN,none,41.0,,none
3,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False,NaN,...,False,NaN,NaN,NaN,Catecholaminergic polymorphic ventricular tach...,NaN,None,16.0,,
4,NaN,False,NaN,NaN,NaN,False,NaN,NaN,True,NaN,...,False,NaN,NaN,NaN,Catecholaminergic polymorphic ventricular tach...,2.0,None,NaN,,
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Catecholaminergic polymorphic ventricular tach...,10.0,None,28.0,,


In [27]:
# make sure all individual_ids are in there
with get_engine() as engine:
    individual_ids = pd.read_sql("SELECT individual_id FROM individual", engine)

set(individual_ids["individual_id"]) - set(condition_info_df.index)

set()

In [29]:
from pathlib import Path

# temp dave this for later
output = Path("../data/04_create_excel_file")

if not output.exists():
    print("Creating output directory")
    output.mkdir()

condition_info_df.to_excel(output / "01_condition_info.xlsx")

Creating output directory


# Family History